## Configuration

In [1]:
!pip install pandas_bokeh
!pip install pgeocode
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.9 MB/s eta 0:00:00


In [2]:
# Importing libraries
import pandas as pd
import sqlite3
import random
import plotly.express as px
import pandas_bokeh
import pgeocode
import plotly.graph_objects as go

from datetime import date, timedelta
from geopandas import GeoDataFrame

pandas_bokeh.output_notebook()

In [3]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def convertSleep(row):
  if row['SleepingTimeFloat'] > 3:
    return row['SleepingTimeFloat'] - 24
  return row['SleepingTimeFloat']

In [4]:
# Cloning gitlab repository
!git clone https://gitlab-student.centralesupelec.fr/tomas.gonzalez-villarroel/projet-s7-groupe-6.git
!cd projet-s7-groupe-6 && git checkout Titouan

# Establishing database connection
datapath = "/content/projet-s7-groupe-6/models/data/BDD.db"
conn = sqlite3.connect(datapath)
cursor = conn.cursor()

# Showing existing tables
query = "SELECT name FROM sqlite_master WHERE type='table'"
cursor.execute(query)
print(cursor.fetchall())
#pd.read_sql_query("PRAGMA table_info(User_Dynam)", conn)

Cloning into 'projet-s7-groupe-6'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 137 (delta 77), reused 62 (delta 31), pack-reused 0
Receiving objects: 100% (137/137), 33.63 KiB | 453.00 KiB/s, done.
Resolving deltas: 100% (77/77), done.
Branch 'Titouan' set up to track remote branch 'Titouan' from 'origin'.
Switched to a new branch 'Titouan'
[('User_Stat',), ('Famille',), ('Visites',), ('Loisirs',), ('User_Dynam',)]


## User Static Attributes

In [5]:
df_stat = pd.read_sql_query("SELECT * FROM User_Stat", conn)
dic_names = df_stat.set_index('UserId').to_dict()["Nom"]
df_stat

,UserId,Nom,Date de naissance,Sexe,Maladies chroniques
0,1234567890123,Albert Dupont,19/12/1975,man,None
1,1111111111111,John Doe,05/01/2002,male,I had chronic pneumonia
2,1234512345123,Mary afton,04/03/1998,female,None


## User Dynamic Attributes

In [6]:
# cursor.execute("""DELETE FROM User_Dynam""")
# conn.commit()

# start_date = date(2022, 1, 1)
# end_date = date(2023, 1, 1)
# prob = [1, 0 , 0, 0, 0 , 0, 0]

# for single_date in daterange(start_date, end_date):
#   if random.choice(prob):
#     sleep_time = str(random.randint(0, 2)).zfill(2) + ':' + str(random.randint(0, 59)).zfill(2)
#   else:
#     sleep_time = str(random.randint(20, 23)).zfill(2) + ':' + str(random.randint(0, 59)).zfill(2)
#   wake_time = str(random.randint(5, 10)).zfill(2) + ':' + str(random.randint(0, 59)).zfill(2)
#   cursor.execute(f"INSERT INTO User_Dynam VALUES ('1234567890123', '{single_date.strftime('%d/%m/%Y')}', 35510, '{sleep_time}', '{wake_time}')")
# conn.commit()

df_dyn = pd.read_sql_query("SELECT * FROM User_Dynam", conn)
df_dyn["SleepingTimeFloat"] = (df_dyn["SleepingTime"].str[:2].astype(int) + df_dyn["SleepingTime"].str[3:].astype(int)/60).round(2)
df_dyn["WakingUpTimeFloat"] = (df_dyn["WakingUpTime"].str[:2].astype(int) + df_dyn["WakingUpTime"].str[3:].astype(int)/60).round(2)
df_dyn["SleepingTimeFloat"] = df_dyn.apply(convertSleep, axis=1)
df_dyn["SleepDuration"] = df_dyn["WakingUpTimeFloat"] - df_dyn["SleepingTimeFloat"]
df_dyn['ChatDate'] = df_dyn["ChatDate"].str[-4:] + '-' + df_dyn["ChatDate"].str[3:5] + '-' + df_dyn["ChatDate"].str[:2]

fig = go.Figure()
df = df_dyn.query("UserID == '1234567890123'")
fig.add_trace(go.Bar(x=df["ChatDate"], y=df["SleepDuration"], base=df["SleepingTimeFloat"]))
fig.update_layout(title=f"{dic_names['1234567890123']}'s sleeping activity", xaxis_title="Date", yaxis_title="Hour of the day")
fig.show()

## User Leisure Activities

In [12]:
# cursor.execute("""DELETE FROM Loisirs""")
# conn.commit()

# activity = ['Badminton', 'Walking', 'Television', 'Chess', 'Groceries']
# opinion = ['Horrible', 'Bad', 'Okay', 'Good', 'Excellent']
# prob = [1, 0, 0, 0, 0]

# start_date = date(2022, 1, 1)
# end_date = date(2023, 1, 1)
# for single_date in daterange(start_date, end_date):
#   if random.choice(prob):
#     cursor.execute(f"INSERT INTO Loisirs VALUES (1234567890123, '{single_date.strftime('%d/%m/%Y')}', '{random.choice(activity)}', '{random.choice(opinion)}')")
# conn.commit()

dic_opinion = {'Horrible':1, 'Bad':2, 'Okay':3, 'Good':4, 'Excellent':5}
df_leisure = pd.read_sql_query("SELECT * FROM Loisirs", conn)
df_leisure["Grade"] = df_leisure["Opinion"]
df_leisure.replace({"Grade": dic_opinion}, inplace=True)
df_leisure['Date'] = df_leisure["Date"].str[-4:] + '-' + df_leisure["Date"].str[3:5] + '-' + df_leisure["Date"].str[:2]

# fig = px.scatter(df_leisure.query("UserID == 1234567890123"), x='Date', y="Grade", color="Opinion", symbol="Activité", title=f"{dic_names['1234567890123']}'s leisure activities (graded time series)", 
#                  category_orders={"Activité": ["Badminton", "Chess", "Groceries", "Television", "Walking"], "Opinion": ["Horrible", "Bad", "Okay", "Good", "Excellent"],})
fig = px.scatter(df_leisure.query("UserID == 1234567890123"), x='Date', y="Grade", color="Activité", title=f"{dic_names['1234567890123']}'s leisure activities (graded time series)")
fig.show()

## User Family

In [15]:
# https://stackoverflow.com/questions/69262280/how-to-use-zipcodes-to-create-map-plot-in-python
# https://plotly.com/python/scatter-plots-on-maps/
# https://plotly.com/python/map-configuration/
# https://plotly.com/python-api-reference/generated/plotly.express.scatter_geo.html

# cursor.execute("""DELETE FROM Famille""")
# cursor.execute("""DELETE FROM Visites""")
# conn.commit()

# cursor.execute("INSERT INTO Famille VALUES (1234567890123, 1, 'Louis Dupont', 'brother', 35510, 'M')")
# cursor.execute("INSERT INTO Famille VALUES (1234567890123, 2, 'Adam Dupont', 'son', 35000, 'M')")
# cursor.execute("INSERT INTO Famille VALUES (1234567890123, 3, 'Léo Dupont', 'son', 72000, 'M')")
# cursor.execute("INSERT INTO Famille VALUES (1234567890123, 4, 'Alice Dupont', 'grand daughter', 75007, 'F')")
# cursor.execute("INSERT INTO Famille VALUES (1234567890123, 5, 'Nina Dupont', 'niece', 90000, 'F')")

# cursor.execute("INSERT INTO Visites VALUES (1234567890123, 1, '05/01/2022')")
# cursor.execute("INSERT INTO Visites VALUES (1234567890123, 2, '18/01/2022')")
# cursor.execute("INSERT INTO Visites VALUES (1234567890123, 3, '24/03/2022')")
# cursor.execute("INSERT INTO Visites VALUES (1234567890123, 4, '12/07/2022')")
# cursor.execute("INSERT INTO Visites VALUES (1234567890123, 3, '12/07/2022')")
# cursor.execute("INSERT INTO Visites VALUES (1234567890123, 3, '10/10/2022')")
# cursor.execute("INSERT INTO Visites VALUES (1234567890123, 5, '24/1/2022')")
# cursor.execute("INSERT INTO Visites VALUES (1234567890123, 4, '30/12/2022')")
# conn.commit()

df_fam = pd.read_sql_query("SELECT * FROM Famille", conn)
df_visit = pd.read_sql_query("SELECT * FROM Visites", conn).rename(columns={'familyID':'FamilyID'})
df_visit = df_visit.merge(df_fam, how="left", on=["UserId", "FamilyID"])
df_visit["-"] = 1
df_visit['date'] = df_visit["date"].str[-4:] + '-' + df_visit["date"].str[3:5] + '-' + df_visit["date"].str[:2]

nomi = pgeocode.Nominatim('fr')
df_fam['Latitude'] = (nomi.query_postal_code(df_fam['Localisation'].astype(str).tolist()).latitude)
df_fam['Longitude'] = (nomi.query_postal_code(df_fam['Localisation'].astype(str).tolist()).longitude)

fig = px.scatter_geo(df_fam.query("UserId == 1234567890123"), lat=df_fam["Latitude"], lon=df_fam["Longitude"], projection="orthographic", center={"lat":48.12120, "lon":-1.6030}, hover_name="Nom", color="Relation", title = f"{dic_names['1234567890123']}'s family location")
fig.update_layout(geo = dict(projection_scale=3.5, showcountries=True))
fig.show()

In [16]:
fig = px.bar(df_visit.query("UserId == 1234567890123"), x="date", y="-", color="Relation", title=f"{dic_names['1234567890123']}'s family visits time series",
             hover_data={'Relation':True, 'date':True, 'Nom':True, "-":False})
fig.update_layout(yaxis={'visible': False, 'showticklabels': True})
fig.show()